# 设置比赛所需要的环境
**因为每次重启netobook后都会设置新的环境，利用jupyter脚本可以一键执行，非常方便**
## 其基本过程如下：
- 1. mindspore安装
- 2. mindformers安装
- 3. 模型权重和tokenizer文件准备
- 4. 数据集准备（这一步可以直接下载Mindrecord格式的数据集）
- 5. 开始微调
- 6. 模型合并
- 7. 原有能力评估
- 8. 微调结果测试

## 1.Mindspore安装

In [ ]:
! pip install mindspore==2.3.0RC2

## 2.Mindformers安装

In [ ]:
%%bash
wget https://2024-ascend-innovation-contest-mindspore.obs.cn-southwest-2.myhuaweicloud.com/topic2-finetune/mindformers.zip
unzip mindformers.zip
cd mindformers/
bash build.sh

In [ ]:
%%bash
export  PYTHONPATH="${PYTHONPATH}:/home/ma-user/work/mindformers/"
pip install tiktoken

## 3.模型权重和tokenizer文件准备

In [ ]:
%%bash
cd /home/ma-user/work/
wget https://2024-ascend-innovation-contest-mindspore.obs.cn-southwest-2.myhuaweicloud.com/topic2-finetune/llama3-8B.ckpt
wget https://2024-ascend-innovation-contest-mindspore.obs.cn-southwest-2.myhuaweicloud.com/topic2-finetune/tokenizer.model

## 4.数据集准备

In [ ]:
%%bash
cd /home/ma-user/work/
wget https://2024-ascend-innovation-contest-mindspore.obs.cn-southwest-2.myhuaweicloud.com/topic2-finetune/train.json

In [ ]:
%%bash
cd /home/ma-user/work
wget https://2024-ascend-innovation-contest-mindspore.obs.cn-southwest-2.myhuaweicloud.com/topic2-finetune/train-data-conversation.json

**也可以下载MindRecord数据集**

In [ ]:
%%bash
wget https://2024-ascend-innovation-contest-mindspore.obs.cn-southwest-2.myhuaweicloud.com/topic2-finetune/train-fastchat256-mindrecore.zip
unzip train-fastchat256-mindrecore.zip

**处理自己抽取出的数据集**

In [ ]:
%%bash
cd /home/ma-user/work/
python data_converter.py --data_path /home/ma-user/work/math_problem/90k-train-data-conversation.json --output_path /home/ma-user/work/math_problem/90k-train-data-conversation.json

In [ ]:
%%bash
export PYTHONPATH="${PYTHONPATH}:/home/ma-user/work/mindformers/"
pip install tiktoken
cd /home/ma-user/work/mindformers/research/llama3
python llama_preprocess.py \
--dataset_type qa \
--input_glob /home/ma-user/work/math_problem/110k-train-data-conversation-v3.json \
--model_file /home/ma-user/work/tokenizer.model \
--seq_length 256 \
--output_file /home/ma-user/work/math_problem/110k-train-fastchat256.mindrecord

## 5. 开始微调

### 4卡微调

In [1]:
%%bash
cd /home/ma-user/work/mindformers/research/
bash ../scripts/msrun_launcher.sh \
"llama3/run_llama3.py \
--config /home/ma-user/work/run_llama3_8b_2024-07-31-v1.yaml \
--load_checkpoint /home/ma-user/work/llama3-8B.ckpt \
--auto_trans_ckpt False \
--use_parallel True \
--run_mode finetune \
--train_data /home/ma-user/work/math_problem/110k-train-fastchat256.mindrecord" 4

../scripts/msrun_launcher.sh: line 119: ulimit: max user processes: cannot modify limit: Operation not permitted


Running Command: msrun --worker_num=4    --local_worker_num=4    --master_port=8118    --log_dir=./output/msrun_log    --join=False    --cluster_time_out=600 llama3/run_llama3.py --config /home/ma-user/work/run_llama3_8b_2024-07-31-v1.yaml --load_checkpoint /home/ma-user/work/llama3-8B.ckpt --auto_trans_ckpt False --use_parallel True --run_mode finetune --train_data /home/ma-user/work/math_problem/110k-train-fastchat256.mindrecord
Please check log files in ./output/msrun_log


/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
[WARNING] ME(2955757:281473427976368,MainP

In [2]:
%%bash
cd /home/ma-user/work/mindformers/research/
bash ../scripts/msrun_launcher.sh \
"llama3/run_llama3.py \
--config /home/ma-user/work/run_llama3_8b_2024-07-29-ptuningv2.yaml \
--load_checkpoint /home/ma-user/work/llama3-8B.ckpt \
--auto_trans_ckpt False \
--use_parallel True \
--run_mode finetune \
--train_data /home/ma-user/work/math_problem/90k-train-fastchat256.mindrecord" 4

../scripts/msrun_launcher.sh: line 119: ulimit: max user processes: cannot modify limit: Operation not permitted


Running Command: msrun --worker_num=4    --local_worker_num=4    --master_port=8118    --log_dir=./output/msrun_log    --join=False    --cluster_time_out=600 llama3/run_llama3.py --config /home/ma-user/work/run_llama3_8b_2024-07-29-ptuningv2.yaml --load_checkpoint /home/ma-user/work/llama3-8B.ckpt --auto_trans_ckpt False --use_parallel True --run_mode finetune --train_data /home/ma-user/work/math_problem/90k-train-fastchat256.mindrecord
Please check log files in ./output/msrun_log


/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
[WARNING] ME(7649:281472947482800,MainProc

## 6.模型合并

In [1]:
%%bash
cd /home/ma-user/work/mindformers/
python mindformers/tools/transform_ckpt.py \
--src_ckpt_strategy /home/ma-user/work/mindformers/research/output/strategy/ \
--src_ckpt_dir /home/ma-user/work/mindformers/research/output/checkpoint/ \
--dst_ckpt_dir /home/ma-user/work/mycheckpoint/ \
--prefix "2024-07-30-lora-llama3-"

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


src_ckpt_strategy: /home/ma-user/work/mindformers/research/output/strategy/merged_ckpt_strategy.ckpt
dst_ckpt_strategy: None
src_ckpt_dir: /home/ma-user/work/mindformers/research/output/checkpoint/
dst_ckpt_dir: /home/ma-user/work/mycheckpoint/
prefix: 2024-07-30-lora-llama3-
......Start transform......


[WARNING] ME(21102:281473055338672,MainProcess):2024-07-30-12:02:55.812.921 [mindspore/parallel/_parallel_serialization.py:396] The parameter scale_sense is not in src_strategy.
[WARNING] ME(21102:281473055338672,MainProcess):2024-07-30-12:05:04.406.678 [mindspore/parallel/_parallel_serialization.py:396] The parameter global_step is not in src_strategy.
[WARNING] ME(21102:281473055338672,MainProcess):2024-07-30-12:05:04.964.422 [mindspore/parallel/_parallel_serialization.py:396] The parameter epoch_num is not in src_strategy.
[WARNING] ME(21102:281473055338672,MainProcess):2024-07-30-12:05:04.964.628 [mindspore/parallel/_parallel_serialization.py:396] The parameter step_num is not in src_strategy.
[WARNING] ME(21102:281473055338672,MainProcess):2024-07-30-12:05:04.964.706 [mindspore/parallel/_parallel_serialization.py:396] The parameter loss_scale is not in src_strategy.


......Transform succeed!......


## 7.原有能力评估
**下载评估数据集**

In [ ]:
%%bash
cd /home/ma-user/work/origin_dataset/
wget https://2024-ascend-innovation-contest-mindspore.obs.cn-southwest-2.myhuaweicloud.com/topic2-finetune/squad1.1.zip
unzip squad1.1.zip

**将数据处理为标准格式Mindrecord数据**

In [ ]:
%%bash
cd /home/ma-user/work/mindformers/mindformers/tools/dataset_preprocess/llama/
python squad_data_process.py \
--input_file /home/ma-user/work/origin_dataset//dev-v1.1.json \
--output_file /home/ma-user/work/origin_dataset/squad8192.mindrecord \
--mode eval \
--max_length 8192 \
--tokenizer_type "llama3-8B" > /home/ma-user/work/origin_dataset/test_eval_base.log 2>&1 &

**评估模型的原有能力**

In [ ]:
%%bash
wget https://2024-ascend-innovation-contest-mindspore.obs.cn-southwest-2.myhuaweicloud.com/topic2-finetune/run_llama3_8b_8k_800T_A2_64G_lora_256_base_eval.yaml -P /home/ma-user/work/mindformers/research/llama3/

In [ ]:
%%bash
cd /home/ma-user/work/mindformers/ 
python run_mindformer.py \
--config research/llama3/run_llama3_8b_8k_800T_A2_64G_lora_256_base_eval.yaml \
--eval_dataset_dir /home/ma-user/work/origin_dataset/squad8192.mindrecord \
--run_mode eval \
--load_checkpoint /home/ma-user/work/mycheckpoint/rank_0/2024-07-30-lora-llama3-0.ckpt \
--epochs 1 \
--batch_size 1 \
--use_parallel False \
--device_id 0 | tee /home/ma-user/work/origin_dataset/2024-07-30_log.txt

**这里在测试的时候，因为需要测试5个模型文件，比较耗时，使用4卡机器可以并行测试**

#### 测试模型-1

In [ ]:
%%bash
cd /home/ma-user/work/mindformers/ 
python run_mindformer.py \
--config research/llama3/run_llama3_8b_8k_800T_A2_64G_lora_256_base_eval.yaml \
--eval_dataset_dir /home/ma-user/work/origin_dataset/squad8192.mindrecord \
--run_mode eval \
--load_checkpoint /home/ma-user/work/mycheckpoint/rank_0/2024-07-301-lora-llama3-0.ckpt \
--epochs 1 \
--batch_size 1 \
--use_parallel False \
--device_id 4 | tee /home/ma-user/work/origin_dataset/2024-07-31-0_log.txt

#### 测试模型-2

In [ ]:
%%bash
cd /home/ma-user/work/mindformers/ 
python run_mindformer.py \
--config research/llama3/run_llama3_8b_8k_800T_A2_64G_lora_256_base_eval.yaml \
--eval_dataset_dir /home/ma-user/work/origin_dataset/squad8192.mindrecord \
--run_mode eval \
--load_checkpoint /home/ma-user/work/mycheckpoint/rank_0/2024-07-31-lora-llama3-1.ckpt \
--epochs 1 \
--batch_size 1 \
--use_parallel False \
--device_id 5 | tee /home/ma-user/work/origin_dataset/2024-07-30-1_log.txt

#### 测试模型-3

In [ ]:
%%bash
cd /home/ma-user/work/mindformers/ 
python run_mindformer.py \
--config research/llama3/run_llama3_8b_8k_800T_A2_64G_lora_256_base_eval.yaml \
--eval_dataset_dir /home/ma-user/work/origin_dataset/squad8192.mindrecord \
--run_mode eval \
--load_checkpoint /home/ma-user/work/mycheckpoint/rank_0/2024-07-31-lora-llama3-2.ckpt \
--epochs 1 \
--batch_size 1 \
--use_parallel False \
--device_id 6 | tee /home/ma-user/work/origin_dataset/2024-07-31-2_log.txt

#### 测试模型-4

In [ ]:
%%bash
cd /home/ma-user/work/mindformers/ 
python run_mindformer.py \
--config research/llama3/run_llama3_8b_8k_800T_A2_64G_lora_256_base_eval.yaml \
--eval_dataset_dir /home/ma-user/work/origin_dataset/squad8192.mindrecord \
--run_mode eval \
--load_checkpoint /home/ma-user/work/mycheckpoint/rank_0/2024-07-31-lora-llama3-3.ckpt \
--epochs 1 \
--batch_size 1 \
--use_parallel False \
--device_id 7 | tee /home/ma-user/work/origin_dataset/2024-07-31-3_log.txt

## 微调结果测试

In [ ]:
%%bash
wget https://2024-ascend-innovation-contest-mindspore.obs.cn-southwest-2.myhuaweicloud.com/topic2-finetune/run_llama3_test.py -P /home/ma-user/work/mindformers/research/llama3/

In [ ]:
!wget https://2024-ascend-innovation-contest-mindspore.obs.cn-southwest-2.myhuaweicloud.com/topic2-finetune/run_llama3_8b_8k_800T_A2_64G_lora_256_eval.yaml -P /home/ma-user/work/mindformers/research/llama3/

In [ ]:
%%bash
cd /home/ma-user/work/mindformers/research
python llama3/run_llama3_test.py \
--config llama3/run_llama3_8b_8k_800T_A2_64G_lora_256_eval.yaml \
--run_mode predict \
--use_parallel False \
--load_checkpoint /home/ma-user/work/mindformers/research/output/checkpoint/rank_0/new_lora_checkpoint_0.ckpt \
--vocab_file /home/ma-user/work/tokenizer.model \
--auto_trans_ckpt False \
--input_dir "/home/ma-user/work/math_eval.json" | tee /home/ma-user/work/origin_dataset/2024-07-30_math_problem_dataset_eval_log.txt

### 将数学问题处理为标准数据格式

In [2]:
%%bash
export PYTHONPATH="${PYTHONPATH}:/home/ma-user/work/mindformers/"
pip install tiktoken
cd /home/ma-user/work/mindformers/research/llama3
python llama_preprocess.py \
--dataset_type qa \
--input_glob /home/ma-user/work/math_problem/30k-test-data-conversation.json \
--model_file /home/ma-user/work/tokenizer.model \
--seq_length 256 \
--output_file /home/ma-user/work/math_problem/30k-test-fastchat256.mindrecord

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple


You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python3.9 -m pip install --upgrade pip' command.
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <c

Transformed 30000 records.
Transform finished, output files refer: /home/ma-user/work/math_problem/30k-test-fastchat256.mindrecord


In [3]:
%%bash
cd /home/ma-user/work/mindformers/ 
python run_mindformer.py \
--config research/llama3/run_llama3_8b_math_problem_eval.yaml \
--eval_dataset_dir /home/ma-user/work/math_problem/30k-test-fastchat256.mindrecord \
--run_mode eval \
--load_checkpoint /home/ma-user/work/mycheckpoint/rank_0/2024-07-29-lora-llama3-0.ckpt \
--epochs 1 \
--batch_size 1 \
--use_parallel False \
--device_id 0 | tee /home/ma-user/work/origin_dataset/2024-07-29-mathproblem_log.txt

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


2024-07-30 16:08:40,716 - mindformers[mindformers/tools/utils.py:153] - INFO - set output path to '/home/ma-user/work/mindformers/output'
2024-07-30 16:08:40,739 - mindformers[mindformers/trainer/trainer.py:919] - INFO - Load configs in /home/ma-user/work/mindformers/configs/general/run_general_task.yaml to build trainer.
2024-07-30 16:08:40,740 - mindformers[mindformers/trainer/trainer.py:949] - INFO - ..........Init Config..........
2024-07-30 16:08:40,740 - mindformers[mindformers/core/parallel_config.py:45] - INFO - initial recompute_config from dict: {'recompute': True, 'select_recompute': False, 'parallel_optimizer_comm_recompute': False, 'mp_comm_recompute': True, 'recompute_slice_activation': True}
2024-07-30 16:08:40,740 - mindformers[mindformers/core/parallel_config.py:51] - INFO - initial parallel_config from dict: {'data_parallel': 1, 'model_parallel': 4, 'pipeline_stage': 1, 'use_seq_parallel': False, 'micro_batch_num': 1, 'vocab_emb_dp': True, 'gradient_aggregation_group'

[WARNING] DEVICE(665579,ffff8920f0b0,python):2024-07-30-16:08:41.027.192 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_memory_adapter.cc:95] Initialize] Reserved memory size for other components(536870912) is less than recommend size(1890134528), It may lead to Out Of Memory in HCCL or other components, Please double check context key 'variable_memory_max_size'/'max_device_memory'
[WARNING] ME(665579:281472982380720,MainProcess):2024-07-30-16:09:03.105.907 [mindspore/ops/primitive.py:203] The in_strategy/in_layout of the operator in your network will not take effect in stand_alone mode. This means the the shard function called in the network is ignored. 
If you want to enable it, please use semi auto or auto parallel mode by context.set_auto_parallel_context(parallel_mode=ParallelMode.SEMI_AUTO_PARALLEL or context.set_auto_parallel_context(parallel_mode=ParallelMode.AUTO_PARALLEL)


2024-07-30 16:09:12,688 - mindformers[mindformers/models/llama/llama_transformer.py:468] - INFO - MoE config is None, use normal FFN


[WARNING] ME(665579:281472982380720,MainProcess):2024-07-30-16:09:12.689.834 [mindspore/common/_decorator.py:40] 'Parameter' is deprecated from version 2.3 and will be removed in a future version, use 'add_pipeline_stage' instead.
[WARNING] ME(665579:281472982380720,MainProcess):2024-07-30-16:09:12.689.965 [mindspore/common/parameter.py:806] This interface may be deleted in the future.


2024-07-30 16:09:15,596 - mindformers[mindformers/models/llama/llama_transformer.py:468] - INFO - MoE config is None, use normal FFN
2024-07-30 16:09:18,305 - mindformers[mindformers/models/llama/llama_transformer.py:468] - INFO - MoE config is None, use normal FFN
2024-07-30 16:09:21,014 - mindformers[mindformers/models/llama/llama_transformer.py:468] - INFO - MoE config is None, use normal FFN
2024-07-30 16:09:23,807 - mindformers[mindformers/models/llama/llama_transformer.py:468] - INFO - MoE config is None, use normal FFN
2024-07-30 16:09:26,619 - mindformers[mindformers/models/llama/llama_transformer.py:468] - INFO - MoE config is None, use normal FFN
2024-07-30 16:09:29,374 - mindformers[mindformers/models/llama/llama_transformer.py:468] - INFO - MoE config is None, use normal FFN


/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 41 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


## 作品上传

In [ ]:
import moxing as mox

#上传一个OBS文件夹sub_dir_0，从Notebook上传至OBS
mox.file.copy_parallel('/home/ma-user/work/20240718-output', 'obs://xxx/xxx/result-2')


INFO:root:List OBS time cost: 1.09 seconds.
